In [39]:
import pandas as pd
import numpy as np

In [40]:
import numpy as np

def generate_grid(lat_min, lat_max, lon_min, lon_max, cell_size):
    lat_steps = int((lat_max - lat_min) / (cell_size / 111.32))
    lon_steps = int((lon_max - lon_min) / (cell_size / (111.32 * np.cos(np.radians(lat_min)))))
    
    lat_grid = np.linspace(lat_min, lat_max, lat_steps)
    lon_grid = np.linspace(lon_min, lon_max, lon_steps)
    
    lon_grid, lat_grid = np.meshgrid(lon_grid, lat_grid)
    return lat_grid, lon_grid


In [41]:
# Parámetros de la rejilla
lat_min = 43.125
lat_max = 43.425
lon_min = -3.725
lon_max = -3.25
cell_size = 4  # Tamaño de la celda en km

# Generar la rejilla
lat_grid, lon_grid = generate_grid(lat_min, lat_max, lon_min, lon_max, cell_size)
lon_lat_vector = np.column_stack((lon_grid.ravel(),lat_grid.ravel()))

In [46]:
data = pd.DataFrame(lon_lat_vector,columns=['Longitud','Latitud'],index=[f'Pluviómetro_{i+1}' for i in range(len(lon_lat_vector))])
data.to_csv('E:/GitHub/NEOPRENE/notebooks/Gaussian_Processes/Attributes.csv')

In [ ]:
import numpy as np
import pandas as pd

# Cargar el archivo original
file_path = 'E:/GitHub/NEOPRENE/notebooks/POINT_SPATIAL_DAILY/CAL1/xi_months.csv'
data = pd.read_csv(file_path)

# Crear un nuevo DataFrame con 5082 columnas
num_new_columns = len(lon_lat_vector) - data.shape[1]  # Columnas adicionales necesarias
base_columns = data.columns.tolist()[1:]  # Excluir la columna de índice
new_columns = base_columns * (num_new_columns // len(base_columns)) + base_columns[:num_new_columns % len(base_columns)]

# Generar nombres únicos para las nuevas columnas
new_columns = [f'Pluviómetro_{i+1}' for i in range(len(lon_lat_vector))]  # Excluyendo la columna de índice

# Crear un nuevo DataFrame con las nuevas columnas
new_data_values = np.tile(data.values[:, 1:], (1, (len(lon_lat_vector) // (data.shape[1] - 1)) + 1))[:, :len(lon_lat_vector)]

# Convertir a DataFrame y asignar nombres de columnas
new_data = pd.DataFrame(new_data_values, columns=new_columns)
new_data.insert(0, 'Index', data['Index'])

# Guardar el nuevo DataFrame en un archivo CSV
new_file_path = 'E:/GitHub/NEOPRENE/notebooks/xi_months_expanded.csv'
new_data.to_csv(new_file_path, index=False)

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skgstat import Variogram
from skgstat import models
from pykrige.ok import OrdinaryKriging

# Datos de ejemplo
np.random.seed(42)
n = 100  # número de puntos
x = np.random.uniform(0, 10, n)
y = np.random.uniform(0, 10, n)
z = np.random.uniform(0, 10, n)

# Crear DataFrame con los datos
data = pd.DataFrame({'x': x, 'y': y, 'z': z})

# Crear y ajustar el semivariograma
coords = data[['x', 'y']].values
values = data['z'].values

# Calcular el semivariograma y ajustar automáticamente el mejor modelo
V = Variogram(coords, values, normalize=False, maxlag=10)

# Ajustar automáticamente el modelo del semivariograma
V.fit()
best_model = V.model

print(f"Mejor modelo de semivariograma: {best_model}")

# Realizar la interpolación Kriging
gridx = np.linspace(0, 10, 100)
gridy = np.linspace(0, 10, 100)
ok = OrdinaryKriging(x, y, z, variogram_model=best_model.__name__.lower())
z_kriged, ss = ok.execute('grid', gridx, gridy)

# Graficar el resultado
plt.imshow(z_kriged, origin='lower', extent=(0, 10, 0, 10))
plt.title('Interpolación Kriging')
plt.colorbar(label='z')
plt.show()


ValueError: The theoretical Variogram function best is not understood, please provide the function

In [23]:
best_model

<function skgstat.models.spherical(h, r, c0, b=0.0)>